# Install Dependencies
Install the required packages `ultralytics` and `roboflow` using pip, as they are not pre-installed in standard Colab environments.

In [1]:
!pip install ultralytics roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 60.1 MB/s eta 0:00:00:00:01
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


# Import Libraries
Import necessary Python libraries including `roboflow`, `ultralytics`, `os`, `shutil`, `yaml`, and `glob`.

In [2]:
from roboflow import Roboflow
from ultralytics import YOLO
import os
import shutil
import yaml
from glob import glob

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# Define Helper Functions
Define the `remap_and_copy` function to handle copying images and remapping label class IDs from source datasets to the merged directory.

In [3]:
def remap_and_copy(source_path, dest_path_images, dest_path_labels, target_class_id):
    """
    Copies images and creates remapped label files.
    """
    os.makedirs(dest_path_images, exist_ok=True)
    os.makedirs(dest_path_labels, exist_ok=True)

    # Process images and corresponding labels
    # YOLOv8 datasets usually have 'images' and 'labels' folders
    
    # We look for images in the source
    source_images = glob(os.path.join(source_path, 'images', '*'))
    
    for img_path in source_images:
        # Copy Image
        shutil.copy(img_path, dest_path_images)
        
        # Process Label
        basename = os.path.basename(img_path)
        name_root, _ = os.path.splitext(basename)
        label_file = os.path.join(source_path, 'labels', f"{name_root}.txt")
        
        if os.path.exists(label_file):
            with open(label_file, 'r') as f:
                lines = f.readlines()
            
            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    # Replace original class_id with target_class_id
                    # We keep coordinates/dimensions same
                    new_line = f"{target_class_id} " + " ".join(parts[1:])
                    new_lines.append(new_line)
            
            if new_lines:
                with open(os.path.join(dest_path_labels, f"{name_root}.txt"), 'w') as f:
                    f.write('\n'.join(new_lines))

# Configure Dataset Parameters
Set up the Roboflow API key and define the `dataset_config` list containing workspace, project, version, and target class mapping details.

In [4]:
api_key = "u0leKBS1HrAOJli1hvLI"

if api_key == "YOUR_API_KEY":
    print("Please replace 'YOUR_API_KEY' within the script.")

# MAPPING CONFIGURATION
# (Workspace, Project, Version, Target_Class_ID, Target_Class_Name)
dataset_config = [
    # GLOBAL CLASS 0: Wagon
    ("aispry-ob85t", "wagon-detection-zsnyn", 2, 0, "Wagon"),
    ("alisha-nyb7f", "wagon-detection-qxlxh", 1, 0, "Wagon"),
    ("wagons-thdfd", "cv-alt", 2, 0, "Wagon"),
    
    # GLOBAL CLASS 1: Wagon parts
    ("db-rail", "train-wagon-cv-project", 3, 1, "Wagon parts"),
    
    # GLOBAL CLASS 2: Wagon numbers
    ("sedykh-marat-dxrw3", "wagon-numbers-detection", 1, 2, "Wagon numbers"),
    ("student-ih3dc", "wagon-detection-qc7bh", 1, 2, "Wagon numbers"),
]

# Download and Merge Datasets
Iterate through the configuration list to download datasets via Roboflow, create the directory structure for `railway_hackathon_merged`, and apply the `remap_and_copy` function to merge train, valid, and test splits.

In [5]:
rf = Roboflow(api_key=api_key)

# Create Merged Dataset Structure
MERGED_DIR = "railway_hackathon_merged"
for split in ['train', 'valid', 'test']:
    os.makedirs(os.path.join(MERGED_DIR, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(MERGED_DIR, split, 'labels'), exist_ok=True)

print("-" * 60)
print("STEP 1: Downloading & Merging Datasets")
print("-" * 60)

for workspace, project_id, version, target_id, target_name in dataset_config:
    try:
        print(f"Processing {workspace}/{project_id} v{version} -> Class {target_id} ({target_name})")
        project = rf.workspace(workspace).project(project_id)
        dataset = project.version(version).download("yolov8")
        
        location = dataset.location
        
        # Merge Train, Valid, Test splits
        for split in ['train', 'valid', 'test']:
            # Some datasets might use 'train' or 'valid' folders differently, standardizing here
            src_split_path = os.path.join(location, split)
            if not os.path.exists(src_split_path): 
                # Try fallback if Roboflow structure varies
                continue
                
            dest_images = os.path.join(MERGED_DIR, split, 'images')
            dest_labels = os.path.join(MERGED_DIR, split, 'labels')
            
            remap_and_copy(src_split_path, dest_images, dest_labels, target_id)
            
    except Exception as e:
        print(f"Skipping {project_id}: {e}")

------------------------------------------------------------
STEP 1: Downloading & Merging Datasets
------------------------------------------------------------
Processing aispry-ob85t/wagon-detection-zsnyn v2 -> Class 0 (Wagon)
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Wagon-detection-2 in yolov8:: 100%|██████████| 4454/4454 [00:00<00:00, 6090.59it/s]


Processing alisha-nyb7f/wagon-detection-qxlxh v1 -> Class 0 (Wagon)
loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov8 in progress : 85.0%
Version export complete for yolov8 format



Extracting Dataset Version Zip to wagon-detection-1 in yolov8:: 100%|██████████| 62/62 [00:00<00:00, 4189.51it/s]

Processing wagons-thdfd/cv-alt v2 -> Class 0 (Wagon)
loading Roboflow workspace...


...


Extracting Dataset Version Zip to Train-Wagon-CV-Project-3 in yolov8:: 100%|██████████| 5442/5442 [00:00<00:00, 7602.21it/s]


Processing sedykh-marat-dxrw3/wagon-numbers-detection v1 -> Class 2 (Wagon numbers)
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Wagon-numbers-detection-1 in yolov8:: 100%|██████████| 376/376 [00:00<00:00, 6343.50it/s]

Processing student-ih3dc/wagon-detection-qc7bh v1 -> Class 2 (Wagon numbers)
loading Roboflow workspace...


loading Roboflow project...


# Create Training Configuration (YAML)
Generate the `data.yaml` file required by YOLO, specifying absolute paths to the merged dataset images and defining class names.

In [6]:
# Create Custom data.yaml
yaml_content = {
    'train': os.path.abspath(os.path.join(MERGED_DIR, 'train', 'images')),
    'val': os.path.abspath(os.path.join(MERGED_DIR, 'valid', 'images')),
    'test': os.path.abspath(os.path.join(MERGED_DIR, 'test', 'images')),
    'nc': 3,
    'names': ['Wagon', 'Wagon parts', 'Wagon numbers']
}

yaml_path = os.path.join(MERGED_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(yaml_content, f)

print("-" * 60)
print(f"Dataset Merged at: {MERGED_DIR}")

------------------------------------------------------------
Dataset Merged at: railway_hackathon_merged


# Train YOLO Model
Initialize the YOLO model (e.g., `yolov8n.pt`) and execute the training process using the generated `data.yaml` file with specified hyperparameters (epochs, batch size, etc.).

In [7]:
print("STEP 2: Starting Training on Merged Dataset")
print("-" * 60)

# Train
model = YOLO('yolov8n.pt') 

try:
    results = model.train(
        data=yaml_path,
        epochs=50,
        imgsz=640,
        batch=-1,
        project='railway_hackathon_take2',
        name='merged_model_v1',
        freeze=10,
        lr0=0.001,
        patience=10
    )
    print("Training Complete!")
except Exception as e:
    print(f"Training Failed: {e}")

STEP 2: Starting Training on Merged Dataset
------------------------------------------------------------
Ultralytics 8.3.238 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=railway_hackathon_merged/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=merged_model_v1, nbs=64, nms

: 

: 

: 

In [1]:
import os
from google.colab import files
import shutil

# Path based on your logs
weights_path = '/content/railway_hackathon_take2/merged_model_v1/weights/best.pt'
results_folder = '/content/railway_hackathon_take2/merged_model_v1'

if os.path.exists(weights_path):
    print(f"✅ Model found at: {weights_path}")
    
    # Option 1: Download just the weights (faster)
    print("Downloading best.pt...")
    files.download(weights_path)
    
    # Option 2: Zip the full results (logs, graphs, weights) and download
    print("Zipping full results folder...")
    shutil.make_archive('training_results', 'zip', results_folder)
    files.download('training_results.zip')
    
else:
    print("❌ File not found. If the Runtime completely disconnected/reset, the files might be lost.")

✅ Model found at: /content/railway_hackathon_take2/merged_model_v1/weights/best.pt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Zipping full results folder...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:
# ...existing code...
from google.colab import drive
import shutil
import os

# 1. Mount Google Drive
# This will ask for permission to access your Drive
drive.mount('/content/drive')

# 2. Define source and destination
source_folder = '/content/railway_hackathon_take2'
destination_zip = '/content/drive/MyDrive/railway_hackathon_results.zip'

# 3. Zip the folder and save to Drive
print("Zipping and saving to Google Drive...")
shutil.make_archive('/content/temp_results', 'zip', source_folder)
shutil.move('/content/temp_results.zip', destination_zip)

print(f"✅ Success! Files saved to: {destination_zip}")


ValueError: mount failed

In [ ]:
# ...existing code...
from google.colab import drive
import shutil
import os

print("🔄 Attempting to force mount Google Drive...")
try:
    # force_remount=True fixes many 'mount failed' errors
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"⚠️ Drive mount failed again: {e}")
    print("Trying to proceed anyway in case it's already attached...")

# Define paths
source_folder = '/content/railway_hackathon_take2'
destination_zip = '/content/drive/MyDrive/railway_hackathon_results.zip'
temp_zip = '/content/temp_results'

if os.path.exists(source_folder):
    print("📦 Zipping training data...")
    # Create zip in local colab space first
    shutil.make_archive(temp_zip, 'zip', source_folder)
    
    if os.path.exists('/content/drive/MyDrive'):
        print(f"🚀 Moving zip to Google Drive: {destination_zip}")
        shutil.copy(f'{temp_zip}.zip', destination_zip)
        print("✅ Success! Check your Google Drive# filepath: c:\Users\dhruv\OneDrive\Desktop\Motion blur mitigation\full model\YOLO\train_yolo_colab.ipynb
# ...existing code...
from google.colab import drive
import shutil
import os

print("🔄 Attempting to force mount Google Drive...")
try:
    # force_remount=True fixes many 'mount failed' errors
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print(f"⚠️ Drive mount failed again: {e}")
    print("Trying to proceed anyway in case it's already attached...")

# Define paths
source_folder = '/content/railway_hackathon_take2'
destination_zip = '/content/drive/MyDrive/railway_hackathon_results.zip'
temp_zip = '/content/temp_results'

if os.path.exists(source_folder):
    print("📦 Zipping training data...")
    # Create zip in local colab space first
    shutil.make_archive(temp_zip, 'zip', source_folder)
    
    if os.path.exists('/content/drive/MyDrive'):
        print(f"🚀 Moving zip to Google Drive: {destination_zip}")
        shutil.copy(f'{temp_zip}.zip', destination_zip)
        print("✅ Success! Check your Google Driveroot folder.")
    else:
        print("❌ Google Drive is not accessible. Cannot copy files.")
else:
    print(f"❌ Source folder not found: {source_folder}").")

SyntaxError: unterminated string literal (detected at line 27) (ipython-input-624446435.py, line 27)